Euclid GC Fisher Matrix Code

Author: Alkistis Pourtsidou, ICG Portsmouth

Using part of http://camb.readthedocs.io/en/latest/CAMBdemo.html 

To run this Jupyter notebook you need to have CAMB and the CAMB python package 
installed. In order to install the CAMB python package on your computer follow
the instructions in http://camb.readthedocs.io/en/latest/

In [2]:
%matplotlib inline
import sys, platform, os

from matplotlib import pyplot as plt
import numpy as np

print('Using CAMB installed at '+ os.path.realpath(os.path.join(os.getcwd(),'..')))
sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
import camb
from camb import model, initialpower

Using CAMB installed at /Users/alkistis/workspace


In [3]:
import scipy
from scipy.interpolate import interp1d
from __future__ import division

In [4]:
from scipy import integrate
from scipy import linalg

pi=np.pi

In [5]:
#Fiducial cosmological parameters

hubble=0.67
omegab=0.022445/hubble**2
omegac=0.121203/hubble**2
om0=omegac+omegab
H00=100*hubble
Ass=2.1265e-9
nss=0.96

kpiv=0.05/hubble #in units [h/Mpc]

gamma=0.545

c=3e5

#print om0

In [6]:
#Set up the fiducial cosmology
pars = camb.CAMBparams()
#Set cosmology
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

In [7]:
#calculate results for these parameters
results = camb.get_results(pars)
#print pars

In [8]:
#Get matter power spectrum at z=0: P(k,z=0)

#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)

#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 200)

#print results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 200)[2]

In [9]:
#Construct P(k,z=0) interpolating function
Pkz0 = interp1d(kh, pk[0])

print Pkz0(0.1)

5714.36444785


In [10]:
#get fiducial σ8 (at z=0)
sig8_fid = results.get_sigma8()[0]
print sig8_fid

0.830287480109


In [11]:
#Redshift bins
zlist = np.arange(0.7,2.1,0.1)
ztest = zlist[0]
Nzbins = len(zlist)

biaslist = [1.083, 1.125, 1.104, 1.126, 1.208, 1.243, 1.282, 1.292, 1.363, 1.497, 1.486, \
            1.491, 1.573, 1.568]

kmin = 0.001
kmax = 0.2

dn3 = [2434.28, 4364.812, 4728.559, 4825.798, 4728.797, 4507.625, 4269.851, 3720.657, 3104.309, \
       2308.975, 1541.831, 1474.707, 893.716, 497.613]

print zlist
print len(zlist), len(dn3)

[ 0.7  0.8  0.9  1.   1.1  1.2  1.3  1.4  1.5  1.6  1.7  1.8  1.9  2. ]
14 14


In [12]:
#Spatially flat Universe

#Define E(z) = H(z)/H0
def Ez(zc):
    return np.sqrt(1-om0+om0*pow(1+zc,3))
def Hz(zc):
    return Ez(zc)*H00

#Define the cosmological distances
def drdz(zp):
    return (c/H00)/Ez(zp)
def rcom(zc):
    return scipy.integrate.romberg(drdz,0,zc)
def DA(zc):
    return rcom(zc)/(1+zc)

print rcom(ztest)
print DA(ztest)

2596.15131772
1527.14783395


In [13]:
#LCDM growth rate
def fg(zz):
    omz=om0*pow(1+zz,3)/pow(Ez(zz),2)
    return pow(omz,gamma)

print fg(ztest)

0.822099034946


In [14]:
#Get the growth factor 
def Dg_dz(zz):
    return -fg(zz)/(1+zz)

def Dgz(zc):
    ans = scipy.integrate.romberg(Dg_dz, 0.0, zc)
    return np.exp(ans)

print Dgz(ztest), Dgz(0)

0.693913449649 1.0


In [15]:
#Construct P_gg(k,μ,z)
def Pgg(kk,mu,zc):
    return pow(bg+fg(zc)*mu**2,2)*pow(Dgz(zc),2)*Pkz0(kk)

In [16]:
#Euclid
Area=15000.0 #deg^2
omegatot = Area*pow(pi/180,2)

Dzbin = 0.1

In [17]:
def photoz(kk,mu,zc):
    sigz = 0.001*(1+zc)
    return np.exp(-pow(kk*mu,2)*pow(c*sigz,2)/pow(Hz(zc),2))

print photoz(0.5,0.5,ztest)

0.200298557962


In [18]:
#survey (bin) volume [Mpc^3]
def dVsurdz(zz):    
    return omegatot*c*pow(rcom(zz),2)/(H00*Ez(zz))
    
def Vsur(zc):
    return scipy.integrate.romberg(dVsurdz,zc-Dzbin/2,zc+Dzbin/2)

def Pshot(zc):
    return Vsur(zc)/Ngal

#effective volume; the hubble**3 factors are for units consistency
def Veff(kk,mu,zc):
    return hubble**3*Vsur(zc)*(Pgg(kk,mu,zc)*photoz(kk,mu,zc)/(Pgg(kk,mu,zc)*photoz(kk,mu,zc)\
                                                     +hubble**3*Pshot(zc)))**2

print ztest, "%.4g" %  Vsur(ztest)

0.7 9.184e+09


In [43]:
#Fisher matrix parameters 
params = ["0:lnH","1:lnDA","2:lnfsig8","3:lnbsig8","4:Ps","5:ns","6:wb","7:wm"]

In [33]:
#Fisher matrix derivatives 

#z-dependent parameters

def dlnP_dlnfsig8(kk,mu,zc):
    return 2*mu**2*fg(zc)/(bg+mu**2*fg(zc))

def dlnP_dlnbsig8(kk,mu,zc):
    return 2*bg/(bg+mu**2*fg(zc))

def dlnP_dlnDA(kk,mu,zc):
    dk = (kmax-kmin)/400
    return -2.0+4*mu**2*(1-mu**2)*fg(zc)/(bg+mu**2*fg(zc))\
            -kk*(1-mu**2)*(1/Pkz0(kk))*(Pkz0(kk+dk)-Pkz0(kk-dk))/(2*dk)

def dlnP_dlnH(kk,mu,zc):
    dk = (kmax-kmin)/400
    return 1.0+4*mu**2*(1-mu**2)*fg(zc)/(bg+mu**2*fg(zc))\
            +kk*mu**2*(1/Pkz0(kk))*(Pkz0(kk+dk)-Pkz0(kk-dk))/(2*dk)

def dlnP_dPs(kk,mu,zc):
    return 1.0/(Pgg(kk,mu,zc)*photoz(kk,mu,zc))

In [40]:
#shape parameters

# Numerical derivative wrt ns 

step_ns = 0.05*nss
ns_p = nss+step_ns
ns_m = nss-step_ns

#DERIV PLUS
pars = camb.CAMBparams()
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=ns_p, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);
results = camb.get_results(pars)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 200)

sig8_ns_p = results.get_sigma8()[0]
Pkz0_ns_p = interp1d(kh, pk[0])

#DERIV MINUS
pars = camb.CAMBparams()
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=ns_m, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);
results = camb.get_results(pars)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 200)

sig8_ns_m= results.get_sigma8()[0]
Pkz0_ns_m = interp1d(kh, pk[0])

#RESET FIDUCIAL
pars = camb.CAMBparams()
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);
results = camb.get_results(pars)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)

#print sig8_ns_m, sig8_fid, sig8_ns_p
#print results.get_sigma8()[0]

# Numerical derivative wrt Ωbh2 

wb = omegab*pow(hubble,2)
step_wb = 0.05*wb
wb_p = wb+step_wb
wb_m = wb-step_wb

#DERIV PLUS
pars = camb.CAMBparams()
pars.set_cosmology(H0=H00, ombh2=wb_p, omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);
results = camb.get_results(pars)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 200)

sig8_wb_p = results.get_sigma8()[0]
Pkz0_wb_p = interp1d(kh, pk[0])

#DERIV MINUS
pars = camb.CAMBparams()
pars.set_cosmology(H0=H00, ombh2=wb_m, omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);
results = camb.get_results(pars)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 200)

sig8_wb_m= results.get_sigma8()[0]
Pkz0_wb_m = interp1d(kh, pk[0])

#RESET FIDUCIAL
pars = camb.CAMBparams()
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);
results = camb.get_results(pars)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)

#print sig8_wb_m, sig8_fid, sig8_wb_p
#print results.get_sigma8()[0]

# Numerical derivative wrt Ωmh2 

wm = omegab*pow(hubble,2)+omegac*pow(hubble,2)
wc = omegac*pow(hubble,2)
step_wm = 0.05*wc
wm_p = wm+step_wm
wm_m = wb-step_wm

#DERIV PLUS
pars = camb.CAMBparams()
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=wc+step_wm,omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);
results = camb.get_results(pars)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 200)

sig8_wm_p = results.get_sigma8()[0]
Pkz0_wm_p = interp1d(kh, pk[0])

#DERIV MINUS
pars = camb.CAMBparams()
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=wc-step_wm,omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);
results = camb.get_results(pars)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 200)

sig8_wm_m= results.get_sigma8()[0]
Pkz0_wm_m = interp1d(kh, pk[0])

#RESET FIDUCIAL
pars = camb.CAMBparams()
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);
results = camb.get_results(pars)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)

print sig8_wm_m, sig8_fid, sig8_wm_p
print results.get_sigma8()[0]

0.804010274594 0.830287480109 0.855539429181
0.830287480109


In [41]:
def dlnP_dns(kk):
    return ((Pkz0_ns_p(kk)/pow(sig8_ns_p,2))-(Pkz0_ns_m(kk)/pow(sig8_ns_m,2)))/(2*step_ns)/(Pkz0(kk)/pow(sig8_fid,2))

def dlnP_dwb(kk):
    return ((Pkz0_wb_p(kk)/pow(sig8_wb_p,2))-(Pkz0_wb_m(kk)/pow(sig8_wb_m,2)))/(2*step_wb)/(Pkz0(kk)/pow(sig8_fid,2))

def dlnP_dwm(kk):
    return ((Pkz0_wm_p(kk)/pow(sig8_wm_p,2))-(Pkz0_wm_m(kk)/pow(sig8_wm_m,2)))/(2*step_wm)/(Pkz0(kk)/pow(sig8_fid,2))

print dlnP_dns(0.1), dlnP_dwb(0.1), dlnP_dwm(0.1)

-0.459308370485 1.15873078798 -0.698583655411


In [42]:
def dF(kk,mu):
    return (1./(8*pi*pi))*pow(kk,2)*deriv_i(kk,mu,zc)*deriv_j(kk,mu,zc)*Veff(kk,mu,zc)    

In [44]:
#2D integration function
def integrate2D(dfun, kgrid, mugrid):
    
    muint = [scipy.integrate.simps(dfun.T[i], mugrid) for i in range(kgrid.size)]
    return scipy.integrate.simps(muint, kgrid)

In [45]:
mugrid = np.linspace(-1., 1., 200) 

In [47]:
fo = open("errors.txt", "w")

#%%time
#   Fisher matrix   # 

Npar = len(params)
#create array of zeros
s = (Npar,Npar)

for zi in range(0,Nzbins):
    
    zc = zlist[zi]
    bg = biaslist[zi]
    zmin = zc-Dzbin/2
    zmax = zc+Dzbin/2
     
    #dVdz = (4*pi/3)*((1+zmax)**3*DA(zmax)**3-(1+zmin)**3*DA(zmin)**3)/(4*pi*pow(180/pi,2))
    #nbar = Dzbin*dn3[zi]/dVdz
    Ngal = dn3[zi]*Area*Dzbin
    #print Ngal
    
    kgrid = np.linspace(kmin, kmax, 400)
    K, MU = np.meshgrid(kgrid, mugrid)
    Fishermat = np.zeros(s)
    for i in range(0,Npar):
        def deriv_i(kk,mu,zc):
            if i==0:  return dlnP_dlnH(kk,mu,zc)
            elif i==1:  return dlnP_dlnDA(kk,mu,zc)
            elif i==2:  return dlnP_dlnfsig8(kk,mu,zc)
            elif i==3:  return dlnP_dlnbsig8(kk,mu,zc)
            elif i==4:  return dlnP_dPs(kk,mu,zc)
            elif i==5:  return dlnP_dns(kk)
            elif i==6:  return dlnP_dwb(kk)
            elif i==7:  return dlnP_dwm(kk)
            else: print "Error: index out of range"
        for  j in range(0,Npar):
            if j>=i:
                def deriv_j(kk,mu,zc):
                    if j==0:  return dlnP_dlnH(kk,mu,zc)
                    elif j==1:  return dlnP_dlnDA(kk,mu,zc)
                    elif j==2:  return dlnP_dlnfsig8(kk,mu,zc)
                    elif j==3:  return dlnP_dlnbsig8(kk,mu,zc)
                    elif j==4:  return dlnP_dPs(kk,mu,zc)
                    elif j==5:  return dlnP_dns(kk)
                    elif j==6:  return dlnP_dwb(kk)
                    elif j==7:  return dlnP_dwm(kk)
                    else: print "Error: index out of range" 
                Fishermat[i][j] = integrate2D(dF(K,MU),kgrid,mugrid)                
            else: Fishermat[i,j] = Fishermat[j,i]
            #if zi==0: print i,j,Fishermat[i][j]
            
    zbin = str(zc)
    lnDerr = str("%e" % np.sqrt(linalg.inv(Fishermat)[1,1]))
    lnHerr = str("%e" % np.sqrt(linalg.inv(Fishermat)[0,0]))
    lnfsig8err = str("%e" % np.sqrt(linalg.inv(Fishermat)[2,2]))
    lnbsig8err = str("%e" % np.sqrt(linalg.inv(Fishermat)[3,3]))
    Pserr = str("%e" % np.sqrt(linalg.inv(Fishermat)[4,4]))  
    nserr = str("%e" % np.sqrt(linalg.inv(Fishermat)[5,5]))
    wberr = str("%e" % np.sqrt(linalg.inv(Fishermat)[6,6]))
    wmerr = str("%e" % np.sqrt(linalg.inv(Fishermat)[7,7]))
    
    print zbin, lnDerr, lnHerr, lnfsig8err, lnbsig8err, Pserr, nserr, wberr, wmerr

    
    outp = zbin + "\t" + lnDerr + "\t" + lnHerr + "\t" + lnfsig8err + "\t" + lnbsig8err + \
    "\t" + Pserr + "\t" + nserr + "\t" + wberr + "\t" + wmerr + "\n"
    fo.write(outp)
    
fo.close()    

0.7 3.468328e-02 3.522367e-02 2.847262e-02 2.808188e-02 3.521302e+01 4.309632e-02 2.514742e-03 1.023965e-02
0.8 2.910476e-02 2.960705e-02 2.407874e-02 2.373060e-02 3.000969e+01 3.719436e-02 2.115503e-03 8.657134e-03
0.9 2.834658e-02 2.879493e-02 2.307210e-02 2.313396e-02 2.589923e+01 3.589283e-02 2.059195e-03 8.411338e-03
1.0 2.783361e-02 2.825954e-02 2.262451e-02 2.269230e-02 2.411429e+01 3.496098e-02 2.020375e-03 8.239883e-03
1.1 2.719136e-02 2.762923e-02 2.249779e-02 2.211197e-02 2.448821e+01 3.403162e-02 1.972253e-03 8.037909e-03
1.2 2.737927e-02 2.780999e-02 2.272694e-02 2.222224e-02 2.380475e+01 3.392896e-02 1.983790e-03 8.069347e-03
1.3 2.773124e-02 2.815968e-02 2.313253e-02 2.246362e-02 2.343440e+01 3.403641e-02 2.007168e-03 8.149001e-03
1.4 2.943223e-02 2.984760e-02 2.443041e-02 2.375785e-02 2.301194e+01 3.540860e-02 2.126373e-03 8.599513e-03
1.5 3.119511e-02 3.162927e-02 2.623274e-02 2.506059e-02 2.449180e+01 3.688536e-02 2.249415e-03 9.066217e-03
1.6 3.404671e-02 3.453815e-0

In [ ]:
#Ngal = 4825.798*Area*Dzbin
#print (Vsur(1.0)/Ngal)

In [ ]:
#nbartest = 0.0001*17*hubble**3
#print 1/nbartest

In [ ]:
#print Pshot(1.0)

In [ ]:
#zminn = 1.0-Dzbin/2
#zmaxx = 1.0+Dzbin/2
#dVdztest = (4*pi/3)*((1+zmaxx)**3*DA(zmaxx)**3-(1+zminn)**3*DA(zminn)**3)/(4*pi*pow(180/pi,2))

#nbartest = Dzbin*4825.798/dVdztest
#print 1/nbartest

In [ ]:
#print "z =",zc,"lnH error =", np.sqrt(linalg.inv(Fishermat)[0,0]),\
    #"lnDA error =", np.sqrt(linalg.inv(Fishermat)[1,1]),\
    #"fsig8 error =",np.sqrt(linalg.inv(Fishermat)[2,2]),\
    #"bsig8 error =",np.sqrt(linalg.inv(Fishermat)[3,3]),\
    #"Ps error =",np.sqrt(linalg.inv(Fishermat)[4,4])

In [ ]:
#errorlist = [zc, np.sqrt(linalg.inv(Fishermat)[1,1])]
    
#    for eachitem in errorlist:
#        fo.write(str(eachitem))#+'\n')